In [107]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [108]:
t=pd.DataFrame(pd.read_csv('test.csv'))
T=pd.DataFrame(pd.read_csv('train.csv'))
s=pd.DataFrame(pd.read_csv('store.csv'))

/tmp/ipython-input-108-4051397894.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  T=pd.DataFrame(pd.read_csv('train.csv'))


In [109]:
train=pd.merge(T,s,on='Store')
test=pd.merge(t,s,on='Store')
test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
41084,41085,1112,6,2015-08-01,1.0,0,0,0,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
41085,41086,1113,6,2015-08-01,1.0,0,0,0,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
41086,41087,1114,6,2015-08-01,1.0,0,0,0,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [110]:
train=train.drop(['Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'],axis=1)
test=test.drop(['Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'],axis=1)

In [111]:
tr_tar=train['Sales']
train=train.drop(['Sales','Customers'],axis=1)

In [112]:
num=list(train.select_dtypes(exclude='object').columns)
obj=list(train.select_dtypes(include='object').columns)

In [113]:
train['StateHoliday'] = train['StateHoliday'].astype(str)
num_transformer = SimpleImputer(strategy='constant')
cat_transformer = Pipeline(steps=[
('imputer', SimpleImputer(strategy='most_frequent')),
('onehot', OneHotEncoder(handle_unknown='ignore'))
])
transformer=ColumnTransformer([
    ('num',num_transformer,num),
    ('cat',cat_transformer,obj),
    ('scaler', StandardScaler(), num)
])
model=XGBRegressor()
Pipe=Pipeline([
    ('preprocessor',transformer),
    ('model',model)
])
Pipe.fit(train,tr_tar)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Store', 'DayOfWeek', 'Open',
                                                   'Promo', 'SchoolHoliday',
                                                   'CompetitionDistance',
                                                   'Promo2', 'Promo2SinceWeek',
                                                   'Promo2SinceYear']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore')...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [114]:
test['StateHoliday'] = test['StateHoliday'].astype(str)

In [118]:
Pipe.predict(test)

array([ 5047.716 ,  8226.134 ,  9639.185 , ...,  5897.585 , 21073.328 ,
        6241.3823], dtype=float32)

In [121]:
submission_df = pd.DataFrame({'Id': test['Id'], 'Sales': Pipe.predict(test)})
submission_df

,Id,Sales
0,1,5047.715820
1,2,8226.133789
2,3,9639.184570
3,4,6212.029297
4,5,7486.141602
...,...,...
41083,41084,3428.426270
41084,41085,8375.954102
41085,41086,5897.584961
41086,41087,21073.328125
